In [1]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append(os.environ['CMS_ROOT'])
from cms_modules.utils import dict_from_hdf5
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

In [16]:
%ls ../../combined-experiments/data-methods/validation

ros-rus-learner.py  rus_0pt0002_d2/     rus_0pt004_d2/      rus_0pt4_d2/
ros_0pt0246_d2/     rus_0pt0005_d2/     rus_0pt008_d2/      rus_0pt8_d2/
ros_0pt0492_d2/     rus_0pt0008_d2/     rus_0pt018_d2/      rus_20_d2/
ros_0pt123_d2/      rus_0pt001_d2/      rus_0pt038_d2/      rus_2_d2/
ros_0pt246_d2/      rus_0pt002_d2/      rus_0pt20_d2/       rus_4_d2/
ros_0pt4925_d2/     rus_0pt003_d2/      rus_0pt26_d2/       rus_8_d2/


In [12]:
# get list of paths to methods to calculat stats on
# data_path = '../../partD-experiments/baselines/validation/'
data_path = '../../combined-experiments/data-methods/validation/'
method_dirs = [os.path.join(data_path, x) for x in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, x)) and '.ipynb' not in x]
thresholds_file = 'thresholds.csv'
valid_metrics = 'valid_metrics.hdf5'

# define output path/metrics to score/epoch to use for roc_auc
output_file = 'best_threshold_stats.csv'
epoch_to_eval = 99

### Helper Functions to Create CSV of Stats

In [19]:
metrics = ['threshold', 'roc_auc', 'tpr', 'tnr', 'geometric_mean', 'arithmetic_mean']
measures = ['mean', 'std', 'min', '25%', '75%', 'max']
columns = [(metric + "_" + measure) for metric in metrics for measure in measures]

# create a dataframe containing all runs for a given method
# calculate the statistics for that method
# return results in comma separated list
def get_stats_for_method_runs(method_path):
    data = { key: [] for key in metrics }
    method_results_path = os.path.join(method_path, 'results')
    runs = os.listdir(method_results_path)
    if len(runs) < 10:
        print('method_results_path', method_results_path, len(runs))
#         return ""
    for run in runs:
        thresholds = pd.read_csv(os.path.join(method_results_path, run, thresholds_file))
        subset = thresholds.loc[thresholds['tpr'] > thresholds['tnr']]
        max_gmean_row = subset.iloc[subset['geometric_mean'].argmax()]
        for key, value in max_gmean_row.items():
            data[key].append(value)
        valid_results = dict_from_hdf5(os.path.join(method_results_path, run, valid_metrics))
        data['roc_auc'].append(valid_results['roc_auc'][epoch_to_eval])
    stats = pd.DataFrame(data).describe()
    print(method_results_path, '\n', stats)
    result = [method_path]
    for metric in metrics:
        for measure in measures:
            result.append(str(round(stats[metric][measure], 5)))
    return ','.join(result) + '\n'

# return comma separated list of column titles
def get_stats_header():
    return ",".join(columns) + "\n"

### Calculate Stats

In [20]:
# create header columns
csv_out = "method," + get_stats_header()

# get statistics for each method
for method in method_dirs:
    if 'ros' not in method:
        csv_out += get_stats_for_method_runs(method)

# write to csv file
with open(output_file, 'w') as f:
    f.write(csv_out)

/Users/jujohnson/anaconda3/envs/tf.latest/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


../../combined-experiments/data-methods/validation/rus_0pt001_d2/results 
        threshold    roc_auc        tpr        tnr  geometric_mean  \
count  10.000000  10.000000  10.000000  10.000000       10.000000   
mean    0.360900   0.998098   0.989472   0.987536        0.988505   
std     0.203259   0.000628   0.005029   0.004184        0.004539   
min     0.113000   0.996944   0.981200   0.980360        0.980780   
25%     0.139250   0.997668   0.986840   0.985272        0.986670   
50%     0.435500   0.998210   0.989660   0.986945        0.987845   
75%     0.540000   0.998562   0.993890   0.991525        0.992845   
max     0.586000   0.998968   0.996240   0.992450        0.993800   

       arithmetic_mean  
count        10.000000  
mean          0.988505  
std           0.004539  
min           0.980780  
25%           0.986670  
50%           0.987845  
75%           0.992845  
max           0.993800  
method_results_path ../../combined-experiments/data-methods/validation/rus_4_d

### View Results

In [21]:
pd.read_csv(output_file)

,method,threshold_mean,threshold_std,threshold_min,threshold_25%,threshold_75%,threshold_max,roc_auc_mean,roc_auc_std,roc_auc_min,roc_auc_25%,roc_auc_75%,roc_auc_max,tpr_mean,tpr_std,tpr_min,tpr_25%,tpr_75%,tpr_max,tnr_mean,tnr_std,tnr_min,tnr_25%,tnr_75%,tnr_max,geometric_mean_mean,geometric_mean_std,geometric_mean_min,geometric_mean_25%,geometric_mean_75%,geometric_mean_max,arithmetic_mean_mean,arithmetic_mean_std,arithmetic_mean_min,arithmetic_mean_25%,arithmetic_mean_75%,arithmetic_mean_max
0,../../combined-experiments/data-methods/valida...,0.36090,0.20326,0.113,0.13925,0.54000,0.58600,0.99810,0.00063,0.99694,0.99767,0.99856,0.99897,0.98947,0.00503,0.98120,0.98684,0.99389,0.99624,0.98754,0.00418,0.98036,0.98527,0.99152,0.99245,0.98850,0.00454,0.98078,0.98667,0.99284,0.99380,0.98850,0.00454,0.98078,0.98667,0.99284,0.99380
1,../../combined-experiments/data-methods/valida...,0.00311,0.00872,0.000,0.00000,0.00005,0.02470,0.98563,0.00241,0.98232,0.98401,0.98731,0.98916,0.98449,0.00821,0.97368,0.97885,0.98966,0.99812,0.53067,0.34011,0.03854,0.31889,0.80740,0.97926,0.67806,0.26315,0.19612,0.55980,0.88721,0.98023,0.75758,0.16657,0.51833,0.65287,0.89266,0.98023
2,../../combined-experiments/data-methods/valida...,0.11290,0.11578,0.041,0.04775,0.11825,0.41900,0.99826,0.00071,0.99651,0.99829,0.99857,0.99911,0.98910,0.00317,0.98496,0.98731,0.98872,0.99624,0.98666,0.00606,0.97195,0.98631,0.98871,0.99456,0.98788,0.00437,0.97843,0.98680,0.98872,0.99540,0.98788,0.00436,0.97846,0.98680,0.98872,0.99540
3,../../combined-experiments/data-methods/valida...,0.63240,0.09714,0.477,0.56600,0.69425,0.75300,0.99787,0.00072,0.99639,0.99748,0.99842,0.99858,0.98910,0.00374,0.98120,0.98872,0.99060,0.99436,0.98691,0.00536,0.97368,0.98847,0.98951,0.99059,0.98800,0.00445,0.97744,0.98867,0.99042,0.99200,0.98800,0.00445,0.97744,0.98867,0.99042,0.99200
4,../../combined-experiments/data-methods/valida...,0.00002,0.00002,0.000,0.00000,0.00004,0.00005,0.99333,0.00653,0.98253,0.98763,0.99837,0.99915,0.99286,0.00912,0.97180,0.99248,1.00000,1.00000,0.45475,0.45343,0.00250,0.00789,0.92015,0.99135,0.53369,0.42748,0.05000,0.08461,0.95299,0.99191,0.72381,0.22403,0.50125,0.50394,0.95408,0.99191
5,../../combined-experiments/data-methods/valida...,0.00099,0.00199,0.000,0.00002,0.00067,0.00650,0.99695,0.00287,0.98981,0.99640,0.99856,0.99883,0.99173,0.00658,0.97932,0.98872,0.99577,1.00000,0.68095,0.43321,0.00019,0.29552,0.98016,0.98901,0.73711,0.38120,0.01382,0.47871,0.98395,0.99168,0.83634,0.21508,0.50010,0.64424,0.98397,0.99169
6,../../combined-experiments/data-methods/valida...,0.45200,0.24980,0.180,0.24500,0.59775,0.93000,0.99735,0.00077,0.99629,0.99660,0.99791,0.99832,0.98590,0.00463,0.97932,0.98214,0.98825,0.99436,0.98356,0.00540,0.97413,0.97973,0.98632,0.99217,0.98473,0.00451,0.97929,0.98124,0.98642,0.99327,0.98473,0.00450,0.97929,0.98125,0.98642,0.99327
7,../../combined-experiments/data-methods/valida...,0.00101,0.00303,0.000,0.00000,0.00000,0.00910,0.98786,0.00253,0.98488,0.98622,0.98970,0.99182,0.98329,0.00620,0.97556,0.97932,0.98684,0.99436,0.69010,0.21922,0.29622,0.58749,0.80519,0.98054,0.81235,0.14118,0.54273,0.75924,0.88714,0.98087,0.83670,0.10723,0.64529,0.78435,0.89131,0.98087
8,../../combined-experiments/data-methods/valida...,0.11840,0.12758,0.013,0.04825,0.13775,0.42400,0.99777,0.00175,0.99420,0.99686,0.99889,0.99954,0.98853,0.00744,0.97368,0.98590,0.99201,0.99812,0.98284,0.01461,0.94815,0.97599,0.99127,0.99552,0.98567,0.01055,0.96361,0.98110,0.99200,0.99682,0.98569,0.01052,0.96373,0.98112,0.99200,0.99682
9,../../combined-experiments/data-methods/valida...,0.00110,0.00213,0.000,0.00000,0.00075,0.00600,0.99285,0.00757,0.98046,0.98546,0.99888,0.99910,0.99248,0.00601,0.98308,0.98778,0.99812,1.00000,0.31641,0.46538,0.00010,0.00948,0.75676,0.99117,0.39034,0.42347,0.00997,0.07652,0.80401,0.99182,0.65444,0.23239,0.50005,0.50262,0.87440,0.99182
